# Generate nice looking figures for the poster

In [1]:
from pathlib import Path
from IPython.display import Image
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
import pandas as pd
import xarray as xr

Define the output path for all images.

In [2]:
outfolder = Path("../../figures")
outfolder.mkdir(exist_ok=True)
datafolder = Path("../../data")

## Background image for the poster

In [3]:
background = outfolder / "poster_background.jpg"

In [6]:
%%bash -s $background
gmt begin
gmt figure $1 jpg E120,A                                                                         
gmt grdimage @earth_relief_01m -R-195/-126/10/50 -JM72i -Cocean -I+a60+nt0.5            
gmt end
du -h $1

6.6M	../../figures/poster_background.jpg


gmt: /home/leo/miniconda3/lib/libcurl.so.4: no version information available (required by /home/leo/pkg/lib64/libgmt.so.6)
gmt: /home/leo/miniconda3/lib/libcurl.so.4: no version information available (required by /home/leo/pkg/lib64/libgmt.so.6)
gmt: /home/leo/miniconda3/lib/libcurl.so.4: no version information available (required by /home/leo/pkg/lib64/libgmt.so.6)
gmt: /home/leo/miniconda3/lib/libcurl.so.4: no version information available (required by /home/leo/pkg/lib64/libgmt.so.6)


In [7]:
Image(url=str(background), width=800, embed=False)

## Synthetic data

In [4]:
%%bash -s $outfolder $datafolder

grid=$2/synthetic-san-andreas-2017.nc
up=$grid?up_velocity
east=$grid?east_velocity
north=$grid?north_velocity
samples=$2/synthetic-san-andreas-2017-samples.csv
vector_color="#333333"
basemap_args=-BNE -B2 --MAP_FRAME_TYPE=plain --MAP_FRAME_PEN=thicker,white --MAP_TICK_PEN=white --FONT_ANNOT=white
cpt=synthetic.cpt
proj=-JM5i

gmt begin

    gmt makecpt -Cpolar -T-1/1 -Q > $cpt
    
    gmt figure $1/synthetic-san-andreas-2017-sample pdf,PNG A
    gmt plot $samples -R$up $proj -Sc0.2c -C$cpt
    gmt basemap $basemap_args

    gmt figure $1/synthetic-san-andreas-2017 pdf,PNG A
    gmt grdimage $up $proj -C$cpt -Q -I+n2
    gmt grdvector $east $north -S200 -G$vector_color -W0.02c,$vector_color -Ix8 -Q0.1c+e
    gmt basemap -BNE -B2 --MAP_FRAME_TYPE=plain --MAP_FRAME_PEN=thicker,white --MAP_TICK_PEN=white --FONT_ANNOT=white
    
gmt end
rm $cpt
du -h $1/*.pdf

96K	../../figures/synthetic-san-andreas-2017.pdf
4.0K	../../figures/synthetic-san-andreas-2017-sample.pdf


bash: line 8: -B2: command not found
gmt: /home/leo/miniconda3/lib/libcurl.so.4: no version information available (required by /home/leo/pkg/lib64/libgmt.so.6)
gmt: /home/leo/miniconda3/lib/libcurl.so.4: no version information available (required by /home/leo/pkg/lib64/libgmt.so.6)
gmt: /home/leo/miniconda3/lib/libcurl.so.4: no version information available (required by /home/leo/pkg/lib64/libgmt.so.6)
gmt: /home/leo/miniconda3/lib/libcurl.so.4: no version information available (required by /home/leo/pkg/lib64/libgmt.so.6)
gmt: /home/leo/miniconda3/lib/libcurl.so.4: no version information available (required by /home/leo/pkg/lib64/libgmt.so.6)
psbasemap [ERROR]: Syntax error: Must specify at least one of -A, -B, -D, -L, -T
gmt: /home/leo/miniconda3/lib/libcurl.so.4: no version information available (required by /home/leo/pkg/lib64/libgmt.so.6)
gmt: /home/leo/miniconda3/lib/libcurl.so.4: no version information available (required by /home/leo/pkg/lib64/libgmt.so.6)
gmt: /home/leo/minico

In [5]:
Image(url=str(outfolder / "synthetic-san-andreas-2017-samples.png"), width=500, embed=False)

## Equations


$$
\begin{bmatrix}
G_{ee} & G_{en} & \alpha\ G_{ev} \\
G_{ne} & G_{nn} & \alpha\ G_{nv} \\
\alpha\ G_{ve} & \alpha\ G_{vn} & G_{vv} \\
\end{bmatrix}
\begin{bmatrix}
f_e \\
f_n \\
f_v \\
\end{bmatrix}
=
\begin{bmatrix}
d_e \\
d_n \\
d_v \\
\end{bmatrix}
$$